In [1]:
%pip install langchain langchain-community langchain-huggingface neo4j pandas spacy trans
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Apps > Advanced app settings > App execution aliases.


In [2]:
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

In [34]:
config = {}

with open('config.txt', 'r') as file:
    for line in file:
        if "=" in line:
            key, value = line.split('=', 1)
            config[key.strip()] = value.strip()

uri = config.get('URI')
username = config.get('USERNAME')
password = config.get('PASSWORD')
driver = GraphDatabase.driver(uri, auth=(username, password))
print("Connected to Neo4j database")

Connected to Neo4j database


In [35]:
# Connect using the LangChain wrapper
graph = Neo4jGraph(
    url=uri,
    username=username,
    password=password,
    refresh_schema= False
)
# Ensure the connection is working by running a quick query (optional)
print(graph.query("MATCH (s:Season) RETURN s"))

[{'s': {'season_name': '2021-22'}}, {'s': {'season_name': '2022-23'}}]


In [36]:
import sys
!{sys.executable} -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     --- ------------------------------------ 1.0/12.8 MB 2.2 MB/s eta 0:00:06
     ----- ---------------------------------- 1.8/12.8 MB 2.6 MB/s eta 0:00:05
     -------- ------------------------------- 2.6/12.8 MB 2.8 MB/s eta 0:00:04
     ---------- ----------------------------- 3.4/12.8 MB 3.0 MB/s eta 0:00:04
     ------------ --------------------------- 3.9/12.8 MB 2.9 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 2.9 MB/s eta 0:00:03
     --------------- ------------------------ 5.0/12.8 MB 2.8 MB/s eta 0:00:03
     ------------------ --------------------- 5.8/12.8 MB 2.9 MB/s eta 0:00:03
     -------------------- ------------------- 6.6/12.8 MB 3.0 MB/s eta 0:

In [7]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")
# Load the kb from the graph database (optional enhancement)
def load_fpl_kb(graph: Neo4jGraph) -> dict:
    kb = {
        "players": [],
        "teams": [],
        "positions": ["gk","gkp", "def", "mid", "fwd", "goalkeeper", "defender", "midfielder", "forward"],
        "stats": {}
    }

    # Load players
    player_results = graph.query("MATCH (p:Player) RETURN p.player_name AS name")
    kb["players"] = [record["name"].lower() for record in player_results]

    # Load teams
    team_results = graph.query("MATCH (t:Team) RETURN t.name AS name")
    kb["teams"] = [record["name"].lower() for record in team_results]

    # Load stats mapping
    kb["stats"] = {
        "points": "total_points",
        "goals": "goals_scored",
        "assists": "assists",
        "minutes": "minutes",
        "bonus": "bonus",
        "influence": "influence",
        "creativity": "creativity",
        "threat": "threat",
        "ict": "ict_index",
        "clean sheets": "clean_sheets",
        "form": "form"
    }

    return kb

In [8]:
FPL_KB = load_fpl_kb(graph)

In [9]:
!pip install transformers torch
!pip install sentencepiece



In [10]:
!pip install torch
!pip install transformers

In [22]:
import transformers
import torch
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")




C:\Users\Habiba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Habiba\.cache\huggingface\hub\models--MoritzLaurer--DeBERTa-v3-base-mnli-fever-anli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
X

In [ ]:
INTENTS = {
    "player_performance",        # Season-long stats for a specific player
    "fixture_details",           # Get fixture info for a team in a gameweek
    "best_players_by_metric",    # Top performers for any stat (goals, assists, etc.)
    "compare_players",           # Head-to-head player comparison
    "player_history_gameweek",   # Player performance in specific gameweek
    "recommendation",            # Form-based player recommendations by position
    "team_performance",          # Aggregated team statistics
    "bonus_points_analysis",     # Bonus point leaders
    "top_players_by_position",   # Position-specific leaderboards        
    "compare_teams",             # Head-to-head team comparison
    "differential_picks"         # Low-ownership high-performers
}

In [24]:
def classify_fpl_intent(query: str) -> str:
   
    candidate_labels = list(INTENTS)

    result = classifier(query, candidate_labels)
    
    intent = result['labels'][0]

    return intent

In [25]:
# ---------------- Test Examples ----------------
tests = [
    # player_performance
    "How did Mohamed Salah perform in the 2022-23 season?",
    "Show me Haaland's stats for last season",
    
    # fixture_details
    "When does Liverpool play in gameweek 15?",
    "What are Arsenal's fixtures in GW 20?",
    
    # best_players_by_metric
    "Who are the top 10 goal scorers?",
    "Show me the top 5 players by assists",
    "Top 10 players by total points",
    
    # compare_players
    "Compare Rashford and Saka",
    "How does Haaland compare to Kane in goals?",
    "Who had more assists: De Bruyne or Odegaard?",
    
    # player_history_gameweek
    "How did Haaland perform in gameweek 5?",
    "What did Salah score in GW 10?",
    "Show me Kane's stats for gameweek 15",
    
    # recommendation
    "Which midfielder should I pick based on recent form?",
    "Recommend me a forward in good form",
    "Who are the best defenders to get right now?",
    "Suggest a goalkeeper based on recent performance",
    
    # team_performance
    "How did Liverpool perform as a team in 2022-23?",
    "Show me Manchester City's overall stats",
    "What are Arsenal's total goals and clean sheets?",
    
    # bonus_points_analysis
    "Who are the top bonus point earners?",
    "Show me the top 5 players by bonus points",
    "Which players got the most bonus points?",
    
    # top_players_by_position
    "Who are the top midfielders by points?",
    "Top 5 forwards by goals scored",
    "Best defenders by clean sheets",
    "Show me the top goalkeepers by saves",
    
    # best_team_lineup
    "What would be the best possible team for 2022-23?",
    "Show me the optimal 11 players",
    "Build me the best fantasy team",
    
    # compare_teams
    "When did Liverpool play Manchester City?",
    "Show me the Arsenal vs Tottenham fixtures",
    
    # differential_picks
    "Find me some differential picks based on recent form",
    "Who are some underrated players with high ICT index?",
    "Suggest some hidden gems for my fantasy team"
]


for t in tests:
    print(t, "=>", classify_fpl_intent(t))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


How did Mohamed Salah perform in the 2022-23 season? => player_performance
Show me Haaland's stats for last season => player_performance
When does Liverpool play in gameweek 15? => fixture_details
What are Arsenal's fixtures in GW 20? => fixture_details
Who are the top 10 goal scorers? => player_performance
Show me the top 5 players by assists => recommendation
Top 10 players by total points => best_players_by_metric
Compare Rashford and Saka => compare_players
How does Haaland compare to Kane in goals? => player_performance
Who had more assists: De Bruyne or Odegaard? => player_performance
How did Haaland perform in gameweek 5? => player_performance
What did Salah score in GW 10? => player_performance
Show me Kane's stats for gameweek 15 => player_performance
Which midfielder should I pick based on recent form? => recommendation
Recommend me a forward in good form => recommendation
Who are the best defenders to get right now? => recommendation
Suggest a goalkeeper based on recent perf

In [19]:
ENTITY_LOOKUP = {}

def add_to_lookup(terms, category):
    for item in terms:
        # If it's a dict (like stats), the item is the key, canonical is the value
        if isinstance(terms, dict):
            value = terms[item]
            key = item
        else:
            value = item.title()
            key = item
        
        ENTITY_LOOKUP[key.lower()] = (category, value)
add_to_lookup(FPL_KB["players"], "player")
add_to_lookup(FPL_KB["teams"], "team")
add_to_lookup(FPL_KB["positions"], "position")
add_to_lookup(FPL_KB["stats"], "stat")

In [ ]:
def extract_fpl_entities(query: str) -> dict:
    """
    Extract entities from FPL query with improved accuracy and validation
    """
    doc = nlp(query)
    entities = {
        "stat_type": "total_points",  # Default fallback
        "season": "2022-23",  # Default season
        "limit": 10  # Default limit
    }
    
    query_lower = query.lower()
    
    # ============================================================================
    # STEP 1: Extract using spaCy + lookup
    # ============================================================================
    
    # Track which players/teams we've seen to handle comparisons
    seen_players = []
    seen_teams = []
    
    for token in doc:
        text = token.text.lower()
        lemma = token.lemma_.lower()
        
        match = ENTITY_LOOKUP.get(text) or ENTITY_LOOKUP.get(lemma)
        
        if match:
            category, value = match
            
            # Handle Players
            if category == "player":
                if value not in seen_players:
                    seen_players.append(value)
                    if "player1" not in entities:
                        entities["player1"] = value
                        entities["player_name"] = value
                    elif "player2" not in entities:
                        entities["player2"] = value
            
            # Handle Teams
            elif category == "team":
                if value not in seen_teams:
                    seen_teams.append(value)
                    if "team1" not in entities:
                        entities["team1"] = value
                        entities["team_name"] = value
                    elif "team2" not in entities:
                        entities["team2"] = value
            
            # Handle Positions
            elif category == "position":
                entities["position"] = value
            
            # Handle Stats
            elif category == "stat":
                entities["stat_type"] = value
    
    # ============================================================================
    # STEP 2: Multi-word entity extraction (e.g., "Mohamed Salah")
    # ============================================================================
    
    # Extract multi-word player names
    for player in FPL_KB["players"]:
        if player.lower() in query_lower:
            if player not in seen_players:
                seen_players.append(player)
                if "player1" not in entities:
                    entities["player1"] = player
                    entities["player_name"] = player
                elif "player2" not in entities:
                    entities["player2"] = player
    
    # Extract multi-word team names (e.g., "Manchester City")
    for team in FPL_KB["teams"]:
        if team.lower() in query_lower:
            if team not in seen_teams:
                seen_teams.append(team)
                if "team1" not in entities:
                    entities["team1"] = team
                    entities["team_name"] = team
                elif "team2" not in entities:
                    entities["team2"] = team
    
    # Extract multi-word stats (e.g., "clean sheets")
    for stat_key, stat_value in FPL_KB["stats"].items():
        if stat_key in query_lower:
            entities["stat_type"] = stat_value
            break
    
    # ============================================================================
    # STEP 3: Regex extraction for structured patterns
    # ============================================================================
    
    # Extract Gameweek
    gw_match = re.search(r"(?:gw|gameweek|game week)\s*(\d+)", query_lower)
    if gw_match:
        entities["gw_number"] = int(gw_match.group(1))
    
    # Extract Season (2022-23 format)
    season_match = re.search(r"(20\d{2}-\d{2})", query_lower)
    if season_match:
        entities["season"] = season_match.group(1)
   
    # Extract "top N" or "best N"
    limit_match = re.search(r"(?:top|best|first)\s*(\d+)", query_lower)
    if limit_match:
        entities["limit"] = int(limit_match.group(1))
    
    # Extract current gameweek for recommendations
    if any(word in query_lower for word in ["recommend", "suggest", "current", "now", "right now"]):
        # If no specific GW mentioned, assume current GW context
        if "current_gw" not in entities and "gw_number" in entities:
            entities["current_gw"] = entities["gw_number"]
        elif "current_gw" not in entities:
            entities["current_gw"] = 20  # Default mid-season
    
    # Extract minimum value filters (e.g., "more than 10 points")
    filter_match = re.search(r"(?:more than|over|at least|minimum)\s*(\d+)", query_lower)
    if filter_match:
        entities["filter_value"] = int(filter_match.group(1))
    
    return entities



In [21]:
def get_fpl_cypher_query(intent: str, entities: dict) -> dict:
    """
    Generate Cypher queries for FPL Knowledge Graph based on user intent.
    
    Args:
        intent: The classified user intent
        entities: Dictionary containing extracted entities from user input
    
    Returns:
        Dictionary with 'query' (Cypher query string) and 'params' (parameters dict)
    """
    
    # Default parameters with extracted entities
    params = {
        "player1": entities.get("player1", ""),
        "player2": entities.get("player2", ""),
        "player_name": entities.get("player_name", ""),
        "team_name": entities.get("team_name", ""),
        "team1": entities.get("team1", ""),
        "team2": entities.get("team2", ""),
        "position": entities.get("position", "MID"),
        "season": entities.get("season", "2022-23"),
        "gw_number": entities.get("gw_number", 1),
        "min_value": entities.get("filter_value", 0),
        "current_gw": entities.get("current_gw", 38),
        "limit": entities.get("limit", 10)
    }

    # Get the stat type for dynamic queries
    stat_prop = entities.get("stat_type", "total_points")

    queries = {
        # 1. PLAYER PERFORMANCE (Full Season)
        "player_performance": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player {player_name: $player_name})
            
            RETURN p.player_name AS Player,
                   COUNT(DISTINCT f) AS Appearances,
                   SUM(r.total_points) AS TotalPoints,
                   SUM(r.goals_scored) AS Goals,
                   SUM(r.assists) AS Assists,
                   SUM(r.minutes) AS Minutes,
                   SUM(r.bonus) AS BonusPoints,
                   SUM(r.clean_sheets) AS CleanSheets,
                   ROUND(AVG(r.total_points), 2) AS AvgPoints,
                   ROUND(AVG(r.form), 2) AS AvgForm
        """,

        # 2. FIXTURE DETAILS (Team + Gameweek)
        "fixture_details": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek {GW_number: $gw_number})
                  -[:HAS_FIXTURE]->(f:Fixture)
            MATCH (f)-[:HAS_HOME_TEAM]->(home:Team)
            MATCH (f)-[:HAS_AWAY_TEAM]->(away:Team)
            
            WHERE home.name = $team_name OR away.name = $team_name
            
            RETURN gw.GW_number AS Gameweek,
                   f.fixture_number AS FixtureNumber,
                   home.name AS HomeTeam,
                   away.name AS AwayTeam,
                   f.kickoff_time AS KickoffTime,
                   CASE 
                       WHEN home.name = $team_name THEN 'Home'
                       ELSE 'Away'
                   END AS Venue
            ORDER BY f.fixture_number
        """,

        # 3. BEST PLAYERS BY METRIC (Top performers for any stat)
        "best_players_by_metric": f"""
            MATCH (s:Season {{season_name: $season}})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player)
            MATCH (p)-[:PLAYS_AS]->(pos:Position)
            
            WITH p, pos, SUM(r.{stat_prop}) AS Total, SUM(r.minutes) AS TotalMinutes
            WHERE TotalMinutes > 900
            
            RETURN p.player_name AS Player,
                   pos.name AS Position,
                   Total,
                   TotalMinutes AS Minutes
            ORDER BY Total DESC
            LIMIT $limit
        """,

        # 4. COMPARE TWO PLAYERS (Head-to-head stats)
        "compare_players": f"""
            MATCH (s:Season {{season_name: $season}})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
            MATCH (p1:Player {{player_name: $player1}})-[r1:PLAYED_IN]->(f)
            MATCH (p2:Player {{player_name: $player2}})-[r2:PLAYED_IN]->(f)
            
            WITH p1, p2,
                 SUM(r1.{stat_prop}) AS Stat1,
                 SUM(r2.{stat_prop}) AS Stat2,
                 SUM(r1.total_points) AS Points1,
                 SUM(r2.total_points) AS Points2,
                 SUM(r1.minutes) AS Minutes1,
                 SUM(r2.minutes) AS Minutes2
            
            RETURN p1.player_name AS Player1,
                   Stat1,
                   Points1,
                   Minutes1,
                   p2.player_name AS Player2,
                   Stat2,
                   Points2,
                   Minutes2
        """,

        # 5. PLAYER HISTORY IN A SPECIFIC GAMEWEEK
        "player_history_gameweek": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek {GW_number: $gw_number})
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player {player_name: $player_name})
            MATCH (f)-[:HAS_HOME_TEAM]->(home:Team)
            MATCH (f)-[:HAS_AWAY_TEAM]->(away:Team)
            
            RETURN p.player_name AS Player,
                   gw.GW_number AS Gameweek,
                   home.name AS HomeTeam,
                   away.name AS AwayTeam,
                   r.total_points AS Points,
                   r.goals_scored AS Goals,
                   r.assists AS Assists,
                   r.minutes AS Minutes,
                   r.bonus AS Bonus,
                   r.clean_sheets AS CleanSheets
        """,

        # 6. PLAYER RECOMMENDATION (Form-based by position)
        "recommendation": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player)
            MATCH (p)-[:PLAYS_AS]->(pos:Position {name: $position})
            
            WHERE gw.GW_number >= ($current_gw - 5) 
                  AND gw.GW_number <= $current_gw
            
            WITH p, 
                 AVG(r.form) AS AvgForm,
                 SUM(r.total_points) AS RecentPoints,
                 SUM(r.minutes) AS RecentMinutes
            WHERE RecentMinutes > 200
            
            RETURN p.player_name AS Player,
                   ROUND(AvgForm, 2) AS AvgForm,
                   RecentPoints,
                   RecentMinutes
            ORDER BY AvgForm DESC, RecentPoints DESC
            LIMIT 5
        """,

        # 7. TEAM PERFORMANCE (Aggregated team stats for a season)
        "team_performance": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
            MATCH (f)-[:HAS_HOME_TEAM|HAS_AWAY_TEAM]->(t:Team {name: $team_name})
            MATCH (p:Player)-[r:PLAYED_IN]->(f)
            
            // Find which team the player belongs to (this is a simplification)
            WITH t, f, r, p
            
            RETURN t.name AS Team,
                   COUNT(DISTINCT f) AS Fixtures,
                   SUM(r.total_points) AS TotalPoints,
                   SUM(r.goals_scored) AS TotalGoals,
                   SUM(r.assists) AS TotalAssists,
                   SUM(r.clean_sheets) AS CleanSheets
        """,

        # 8. BONUS POINTS LEADERS (Top bonus point earners)
        "bonus_points_analysis": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player)
            MATCH (p)-[:PLAYS_AS]->(pos:Position)
            
            WITH p, pos, SUM(r.bonus) AS TotalBonus, SUM(r.bps) AS TotalBPS
            WHERE TotalBonus > 0
            
            RETURN p.player_name AS Player,
                   pos.name AS Position,
                   TotalBonus,
                   TotalBPS
            ORDER BY TotalBonus DESC
            LIMIT $limit
        """,

        # 9. TOP PLAYERS BY POSITION (Position-specific leaderboard)
        "top_players_by_position": f"""
            MATCH (s:Season {{season_name: $season}})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player)
            MATCH (p)-[:PLAYS_AS]->(pos:Position {{name: $position}})
            
            WITH p, 
                 SUM(r.{stat_prop}) AS Total,
                 SUM(r.total_points) AS TotalPoints,
                 SUM(r.minutes) AS TotalMinutes
            WHERE TotalMinutes > 500
            
            RETURN p.player_name AS Player,
                   Total,
                   TotalPoints,
                   TotalMinutes
            ORDER BY Total DESC
            LIMIT $limit
        """,

        # 10. BEST TEAM LINEUP (Optimal 11 players based on points)
        "best_team_lineup": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player)
            MATCH (p)-[:PLAYS_AS]->(pos:Position)
            
            WITH p, pos, SUM(r.total_points) AS TotalPoints, SUM(r.minutes) AS TotalMinutes
            WHERE TotalMinutes > 1000
            
            WITH pos.name AS Position, 
                 COLLECT({player: p.player_name, points: TotalPoints}) AS Players
            
            // Get top players per position
            WITH Position, 
                 [player IN Players | player] AS SortedPlayers
            ORDER BY Position
            
            RETURN Position,
                   SortedPlayers[0..3] AS TopPlayers
        """,

        # 11. TEAM HEAD-TO-HEAD (Compare two teams' performance)
        "compare_teams": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
            MATCH (f)-[:HAS_HOME_TEAM]->(t1:Team {name: $team1})
            MATCH (f)-[:HAS_AWAY_TEAM]->(t2:Team {name: $team2})
            MATCH (p:Player)-[r:PLAYED_IN]->(f)
            
            WITH f, t1, t2,
                 SUM(CASE WHEN (p)-[:PLAYED_IN]->(f) THEN r.goals_scored ELSE 0 END) AS Goals
            
            RETURN t1.name AS Team1,
                   t2.name AS Team2,
                   f.fixture_number AS FixtureNumber,
                   gw.GW_number AS Gameweek,
                   f.kickoff_time AS KickoffTime
        """,

        # 12. DIFFERENTIAL PICKS (Low-ownership, high-performing players)
        "differential_picks": """
            MATCH (s:Season {season_name: $season})
                  -[:HAS_GW]->(gw:Gameweek)
                  -[:HAS_FIXTURE]->(f:Fixture)
                  <-[r:PLAYED_IN]-(p:Player)
            MATCH (p)-[:PLAYS_AS]->(pos:Position)
            
            WHERE gw.GW_number >= ($current_gw - 5) 
                  AND gw.GW_number <= $current_gw
            
            WITH p, pos,
                 AVG(r.form) AS AvgForm,
                 SUM(r.total_points) AS RecentPoints,
                 AVG(r.ict_index) AS AvgICT
            WHERE RecentPoints >= $min_value
            
            RETURN p.player_name AS Player,
                   pos.name AS Position,
                   ROUND(AvgForm, 2) AS Form,
                   RecentPoints,
                   ROUND(AvgICT, 2) AS ICTIndex
            ORDER BY AvgICT DESC, RecentPoints DESC
            LIMIT 10
        """
    }

    # Return the selected query or a default fallback
    return {
        "query": queries.get(intent, "MATCH (n) RETURN 'Intent not found' AS message LIMIT 1"),
        "params": params
    }

In [ ]:
from langchain_core.language_models import LLM
from typing import Optional, List, Any
from pydantic import Field

class GemmaLangChainWrapper(LLM):
    client: Any = Field(...)
    max_tokens: int = 500
    
    @property
    def _llm_type(self) -> str:
        return "gemma_hf_api"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.client.chat_completion(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=self.max_tokens,
            temperature=0.2 
        )
        return response.choices[0].message["content"]


c:\Users\abdel\Repos\Fantasy-Premier-League-Companion\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%pip install -U langchain langchain-community langchain-core pydantic typing-extensions

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install langchain langchain-community langchain-core langchain-huggingface

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install langchain-classic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 1. New Imports
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
# Keep these standard imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from huggingface_hub import InferenceClient

HF_TOKEN = config.get('HF_TOKEN')

def initialize_rag_pipeline(query_result: str = ""):
    # --- Setup Vector Store (Same as before) ---
    splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
    documents = splitter.create_documents([query_result])
    
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    client = InferenceClient(
        model="google/gemma-2-2b-it",
        token=HF_TOKEN
    )
    
    gemma_llm = GemmaLangChainWrapper(client=client)
    
    # 1. Create a prompt specifically for the LLM to process chunks
    prompt = ChatPromptTemplate.from_template("""
    Answer the user's question based on the context below:
    
    <context>
    {context}
    </context>

    Question: {input}
    """)

    # 2. Create the "Stuff" chain (combines retrieved docs into the prompt)
    document_chain = create_stuff_documents_chain(gemma_llm, prompt)

    # 3. Create the final Retrieval Chain (connects retriever -> document chain)
    qa_chain = create_retrieval_chain(retriever, document_chain)
    
    return qa_chain

In [ ]:
query = "Who is the top 3 players when it comes to total points in season 2021-22?"
query_result = ""
intent = classify_fpl_intent(query)
print("Intent:", intent)
entities = extract_fpl_entities(query)
print("Entities:", entities)
cypher_info = get_fpl_cypher_query(intent, entities)
print("Cypher Query:", cypher_info["query"])
cypher_query = cypher_info["query"]
print("Cypher Params:", cypher_info["params"])
query_result = graph.query(cypher_query, cypher_info["params"])
print(query_result)

Intent: PLAYER_STATS_TOP
Entities: {'stat_type': 'total_points', 'filter_value': 22.0, 'season': '2021-22'}
Cypher Query: 
            MATCH (p:Player)-[r:PLAYED_IN]->(:Fixture)<-[:HAS_FIXTURE]-(:Gameweek)<-[HAS_GW]-(:Season {season_name: $season})
            RETURN p.player_name, sum(r.total_points) AS TotalStat 
            ORDER BY TotalStat DESC LIMIT 10
        
Cypher Params: {'player1': '', 'player2': '', 'player_name': '', 'team_name': '', 'team1': '', 'team2': '', 'position': 'MID', 'season': '2021-22', 'gw_number': 1, 'min_value': 22.0, 'current_gw': 38}
[{'p.player_name': 'Mohamed Salah', 'TotalStat': 265}, {'p.player_name': 'Heung-Min Son', 'TotalStat': 258}, {'p.player_name': 'Trent Alexander-Arnold', 'TotalStat': 208}, {'p.player_name': 'Jarrod Bowen', 'TotalStat': 206}, {'p.player_name': 'João Pedro Cavaco Cancelo', 'TotalStat': 201}, {'p.player_name': 'Kevin De Bruyne', 'TotalStat': 196}, {'p.player_name': 'Harry Kane', 'TotalStat': 192}, {'p.player_name': 'Andrew Robe

In [ ]:
query_result = str(query_result)
rag_chain = initialize_rag_pipeline(query_result)
response = rag_chain.invoke({"input": query})
print(response["answer"])

Based on the provided context, here are the top 3 players with the highest total points in the 2021-22 season:

1. **Mohamed Salah:** 265 points
2. **Heung-Min Son:** 258 points
3. **Kevin De Bruyne:** 196 points 



In [ ]:
%pip install faiss-gpu

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Test 1: Comparison
user_query_1 = "Who has more goals, Salah or Haaland?"
print(f"Query: {user_query_1}")

ents_1 = extract_fpl_entities(user_query_1)
intent_1 = classify_fpl_intent(user_query_1)
result_1 = get_fpl_cypher_query(intent_1, ents_1)

print(f"Entities: {ents_1}")
print(f"Intent: {intent_1}")
print(f"Cypher Params: {result_1['params']}")
print("-" * 30)

# Test 2: Specific GW Stats
user_query_2 = "How many points did Saka get in Gameweek 5?"
print(f"Query: {user_query_2}")

ents_2 = extract_fpl_entities(user_query_2)
intent_2 = classify_fpl_intent(user_query_2)
result_2 = get_fpl_cypher_query(intent_2, ents_2)

print(f"Entities: {ents_2}")
print(f"Intent: {intent_2}")
print(f"Cypher Params: {result_2['params']}")